Here we'll first be loading the FC matrices and explore their structure

In [2]:
#using read_dataset from Datasets/FC/create_dataset.py to read the dataset
from Datasets.FC.create_dataset import read_dataset

In [3]:
dataset = read_dataset()

In [4]:
#ok, let's explore the data a bit more
#dataset is a list object of torch_geometric.data objects

#let's see the first element
print(dataset[0])

#it's a dictionary object, let's see the keys
print(dataset[0].keys())

#ok, let's see the values
print(dataset[0].values())

#it has 4 keys, 'x', 'edge_index', 'edge_attr' and 'y' where y=0 menas the patient is healthy and y=1 means the patient has Autism Spectrum Disorder (ASD)
"""graph = Data(x=ROI.reshape(-1,116).float(),
                     edge_index=G.indices().reshape(2,-1).long(),
                     edge_attr=G.values().reshape(-1,1).float(),
                     y=y.long())"""

#how much data do we have?
len(dataset)


Data(x=[116, 116], edge_index=[2, 1016], edge_attr=[1016, 1], y=[1])
['x', 'edge_index', 'edge_attr', 'y']
ValuesView({'x': tensor([[ 0.0000,  0.4543,  0.2477,  ...,  0.1753,  0.2247, -0.1751],
        [ 0.4543,  0.0000, -0.2204,  ..., -0.1947, -0.2258, -0.1434],
        [ 0.2477, -0.2204,  0.0000,  ..., -0.0521, -0.0804, -0.2025],
        ...,
        [ 0.1753, -0.1947, -0.0521,  ...,  0.0000,  0.6875, -0.1364],
        [ 0.2247, -0.2258, -0.0804,  ...,  0.6875,  0.0000,  0.0929],
        [-0.1751, -0.1434, -0.2025,  ..., -0.1364,  0.0929,  0.0000]]), 'edge_index': tensor([[  0,   0,   0,  ..., 114, 115, 115],
        [  1,  10,  12,  ..., 113,  94, 109]]), 'edge_attr': tensor([[0.4543],
        [0.5913],
        [0.4224],
        ...,
        [0.6875],
        [0.4846],
        [0.5437]]), 'y': tensor([0])})


1099

In [5]:
#set the seed
import torch
torch.manual_seed(0)

MODEL = "GCN"
DATASET = "FC"

from models.models_FC import GCN_framework as framework # import the model

gnn = framework(dataset,device="cpu")

print(gnn.model)
print(gnn.train_idx)

Net(
  (conv_layers): ModuleList(
    (0): GCNConv(116, 128)
    (1-4): 4 x GCNConv(128, 128)
  )
  (batch_norms): ModuleList(
    (0-4): 5 x BatchNorm(128)
  )
  (lin1): Linear(128, 128, bias=True)
  (lin2): Linear(128, 2, bias=True)
  (bn1): BatchNorm(128)
  (bn2): BatchNorm(2)
)
tensor([217, 137, 426,  ..., 643, 205, 434])


/home/tpelletreaudur/.local/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
#gnn.iterate()

Epoch: 020, Loss: 0.143, Test Loss: 0.793, Train Acc: 0.998 Test Acc: 0.564
Epoch: 040, Loss: 0.124, Test Loss: 0.756, Train Acc: 0.998 Test Acc: 0.564
Epoch: 060, Loss: 0.092, Test Loss: 0.796, Train Acc: 1.000 Test Acc: 0.545
Epoch: 080, Loss: 0.082, Test Loss: 0.794, Train Acc: 1.000 Test Acc: 0.636
Epoch: 100, Loss: 0.076, Test Loss: 0.734, Train Acc: 1.000 Test Acc: 0.600
Epoch: 120, Loss: 0.070, Test Loss: 0.719, Train Acc: 1.000 Test Acc: 0.636
Epoch: 140, Loss: 0.067, Test Loss: 0.724, Train Acc: 1.000 Test Acc: 0.673


In [6]:
#gnn.train()

0.06823100771940531

In [7]:
#save the model 
#gnn.save_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

Model saved in: models/FC_GCNserver.pt


In [6]:
#load the model
gnn.load_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

In [7]:
gnn.evaluate()

Test Loss: 0.724, Train Acc: 1.000 Test Acc: 0.673


In [9]:
train_features, test_features = gnn.evaluate_with_features2()

In [10]:
print(len(train_features[0]))
len(train_features), len(test_features)

8


(1044, 55)

In [11]:
import networkx as nx
def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Alternative metrics for disconnected graphs
        # Option 1: Use the average path length of the largest connected component
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        density = nx.density(G)
        avg_path_len = calculate_avg_path_length(G)
        num_cliques = len(list(nx.find_cliques(G)))
        num_triangles = sum(nx.triangles(G).values()) / 3
        num_squares = sum(nx.square_clustering(G).values()) / 4
        number_of_node_in_the_largest_fully_connected_component = len(max(nx.connected_components(G), key=len))
        #small_world = nx.algorithms.smallworld.sigma(G)

        properties.append((num_nodes, num_edges, density, avg_path_len, num_cliques, num_triangles, num_squares, number_of_node_in_the_largest_fully_connected_component)) #, small_world))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties = compute_graph_properties(selected_dataset)

In [11]:
print(len(train_properties))
train_properties[0:5]



1044


[(116,
  717,
  0.10749625187406296,
  2.6724137931034484,
  263,
  1735.0,
  6.64946643069199,
  116),
 (116,
  473,
  0.07091454272863568,
  3.4743628185907047,
  158,
  708.0,
  7.095858020333574,
  116),
 (116,
  997,
  0.1494752623688156,
  2.469265367316342,
  343,
  3591.0,
  7.864813698203321,
  116),
 (116,
  892,
  0.13373313343328336,
  2.874662668665667,
  144,
  3409.0,
  10.613595627764592,
  116),
 (116,
  504,
  0.07556221889055473,
  3.1734202763546033,
  155,
  861.0,
  6.351955841015705,
  114)]

In [11]:
import pickle as pkl
#save the properties in a file
# with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "wb") as f:
#     pkl.dump(train_properties, f)

# with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "wb") as f:
#     pkl.dump(test_properties, f)

#load the properties
# with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "rb") as f:
#     train_properties = pkl.load(f)

# with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "rb") as f:
#     test_properties = pkl.load(f)

In [12]:
"""
The embeddings of GCN are like this:
return F.log_softmax(x7, dim=-1), (x1, x2, x3, x4, x5, x_global, x6, x7)
"""


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
output_size = 1  # Predicting one property at a time
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Assume we have already evaluated to get features
#train_features, test_features = gnn.evaluate_with_features2()

# Extract x embeddings
train_x = np.array([feat[0] for feat in train_features])
test_x = np.array([feat[0] for feat in test_features])

# Extract 2, 3, 4, global, 5, 6, 7 embeddings
train_x2 = np.array([feat[1] for feat in train_features])
test_x2 = np.array([feat[1] for feat in test_features])

train_x3 = np.array([feat[2] for feat in train_features])
test_x3 = np.array([feat[2] for feat in test_features])

train_x4 = np.array([feat[3] for feat in train_features])
test_x4 = np.array([feat[3] for feat in test_features])

train_x5 = np.array([feat[4] for feat in train_features])
test_x5 = np.array([feat[4] for feat in test_features])

train_x_global = np.array([feat[5] for feat in train_features])
test_x_global = np.array([feat[5] for feat in test_features])

train_x6 = np.array([feat[6] for feat in train_features])
test_x6 = np.array([feat[6] for feat in test_features])

train_x7 = np.array([feat[7] for feat in train_features])
test_x7 = np.array([feat[7] for feat in test_features])

# Compute graph properties
# train_properties = compute_graph_properties(gnn.dataset[gnn.train_idx])
# test_properties = compute_graph_properties(gnn.dataset[gnn.test_idx])

# Convert to PyTorch tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
train_x2 = torch.tensor(train_x2, dtype=torch.float32)
train_x3 = torch.tensor(train_x3, dtype=torch.float32)
train_x4 = torch.tensor(train_x4, dtype=torch.float32)
train_x5 = torch.tensor(train_x5, dtype=torch.float32)
train_x_global = torch.tensor(train_x_global, dtype=torch.float32)
train_x6 = torch.tensor(train_x6, dtype=torch.float32)
train_x7 = torch.tensor(train_x7, dtype=torch.float32)

test_x = torch.tensor(test_x, dtype=torch.float32)
test_x2 = torch.tensor(test_x2, dtype=torch.float32)
test_x3 = torch.tensor(test_x3, dtype=torch.float32)
test_x4 = torch.tensor(test_x4, dtype=torch.float32)
test_x5 = torch.tensor(test_x5, dtype=torch.float32)
test_x_global = torch.tensor(test_x_global, dtype=torch.float32)
test_x6 = torch.tensor(test_x6, dtype=torch.float32)
test_x7 = torch.tensor(test_x7, dtype=torch.float32)

train_y = torch.tensor(train_properties, dtype=torch.float32)
test_y = torch.tensor(test_properties, dtype=torch.float32)

# Train and evaluate a model for each graph property and each embedding
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings = [(train_x, test_x), (train_x2, test_x2), (train_x3, test_x3), (train_x4, test_x4), (train_x5, test_x5), (train_x_global, test_x_global), (train_x6, test_x6), (train_x7, test_x7)]
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']

In [13]:
#create a dictionary where we will sotre the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 500000  # Adjust this as needed

        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Adjust this for more frequent/lower print frequency
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embdedding = embeddings_names[ii]
            results[(name_of_embdedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "wb") as f:
    pkl.dump(results, f)

Epoch [1000/500000], Property: num_nodes, Loss: 11254.4902
Epoch [2000/500000], Property: num_nodes, Loss: 9736.1846
Epoch [3000/500000], Property: num_nodes, Loss: 8486.0703
Epoch [4000/500000], Property: num_nodes, Loss: 7430.9277
Epoch [5000/500000], Property: num_nodes, Loss: 6530.4951
Epoch [6000/500000], Property: num_nodes, Loss: 5757.3501
Epoch [7000/500000], Property: num_nodes, Loss: 5095.4639
Epoch [8000/500000], Property: num_nodes, Loss: 4533.2495
Epoch [9000/500000], Property: num_nodes, Loss: 4056.4666
Epoch [10000/500000], Property: num_nodes, Loss: 3648.5596
Epoch [11000/500000], Property: num_nodes, Loss: 3294.9548
Epoch [12000/500000], Property: num_nodes, Loss: 2986.7925
Epoch [13000/500000], Property: num_nodes, Loss: 2718.5740
Epoch [14000/500000], Property: num_nodes, Loss: 2486.8418
Epoch [15000/500000], Property: num_nodes, Loss: 2288.6736
Epoch [16000/500000], Property: num_nodes, Loss: 2120.0205
Epoch [17000/500000], Property: num_nodes, Loss: 1976.0278
Epoch

Epoch [143000/500000], Property: num_nodes, Loss: 1.6973
Epoch [144000/500000], Property: num_nodes, Loss: 1.6973
Epoch [145000/500000], Property: num_nodes, Loss: 1.6973
Epoch [146000/500000], Property: num_nodes, Loss: 1.6973
Epoch [147000/500000], Property: num_nodes, Loss: 1.6973
Epoch [148000/500000], Property: num_nodes, Loss: 1.6973
Epoch [149000/500000], Property: num_nodes, Loss: 1.6973
Epoch [150000/500000], Property: num_nodes, Loss: 1.6973
Epoch [151000/500000], Property: num_nodes, Loss: 1.6973
Epoch [152000/500000], Property: num_nodes, Loss: 1.6973
Epoch [153000/500000], Property: num_nodes, Loss: 1.6973
Epoch [154000/500000], Property: num_nodes, Loss: 1.6973
Epoch [155000/500000], Property: num_nodes, Loss: 1.6973
Epoch [156000/500000], Property: num_nodes, Loss: 1.6973
Epoch [157000/500000], Property: num_nodes, Loss: 1.6973
Epoch [158000/500000], Property: num_nodes, Loss: 1.6973
Epoch [159000/500000], Property: num_nodes, Loss: 1.6973
Epoch [160000/500000], Property

Epoch [287000/500000], Property: num_nodes, Loss: 1.6973
Epoch [288000/500000], Property: num_nodes, Loss: 1.6973
Epoch [289000/500000], Property: num_nodes, Loss: 1.6973
Epoch [290000/500000], Property: num_nodes, Loss: 1.6973
Epoch [291000/500000], Property: num_nodes, Loss: 1.6973
Epoch [292000/500000], Property: num_nodes, Loss: 1.6973
Epoch [293000/500000], Property: num_nodes, Loss: 1.6973
Epoch [294000/500000], Property: num_nodes, Loss: 1.6973
Epoch [295000/500000], Property: num_nodes, Loss: 1.6973
Epoch [296000/500000], Property: num_nodes, Loss: 1.6973
Epoch [297000/500000], Property: num_nodes, Loss: 1.6973
Epoch [298000/500000], Property: num_nodes, Loss: 1.6973
Epoch [299000/500000], Property: num_nodes, Loss: 1.6973
Epoch [300000/500000], Property: num_nodes, Loss: 1.6973
Epoch [301000/500000], Property: num_nodes, Loss: 1.6973
Epoch [302000/500000], Property: num_nodes, Loss: 1.6973
Epoch [303000/500000], Property: num_nodes, Loss: 1.6973
Epoch [304000/500000], Property

Epoch [431000/500000], Property: num_nodes, Loss: 1.6973
Epoch [432000/500000], Property: num_nodes, Loss: 1.6973
Epoch [433000/500000], Property: num_nodes, Loss: 1.6973
Epoch [434000/500000], Property: num_nodes, Loss: 1.6973
Epoch [435000/500000], Property: num_nodes, Loss: 1.6973
Epoch [436000/500000], Property: num_nodes, Loss: 1.6973
Epoch [437000/500000], Property: num_nodes, Loss: 1.6973
Epoch [438000/500000], Property: num_nodes, Loss: 1.6973
Epoch [439000/500000], Property: num_nodes, Loss: 1.6973
Epoch [440000/500000], Property: num_nodes, Loss: 1.6973
Epoch [441000/500000], Property: num_nodes, Loss: 1.6973
Epoch [442000/500000], Property: num_nodes, Loss: 1.6973
Epoch [443000/500000], Property: num_nodes, Loss: 1.6973
Epoch [444000/500000], Property: num_nodes, Loss: 1.6973
Epoch [445000/500000], Property: num_nodes, Loss: 1.6973
Epoch [446000/500000], Property: num_nodes, Loss: 1.6973
Epoch [447000/500000], Property: num_nodes, Loss: 1.6973
Epoch [448000/500000], Property

Epoch [69000/500000], Property: num_edges, Loss: 62155.3828
Epoch [70000/500000], Property: num_edges, Loss: 61447.6719
Epoch [71000/500000], Property: num_edges, Loss: 60764.7461
Epoch [72000/500000], Property: num_edges, Loss: 60106.4844
Epoch [73000/500000], Property: num_edges, Loss: 59472.0547
Epoch [74000/500000], Property: num_edges, Loss: 58860.7891
Epoch [75000/500000], Property: num_edges, Loss: 58272.0742
Epoch [76000/500000], Property: num_edges, Loss: 57704.9727
Epoch [77000/500000], Property: num_edges, Loss: 57158.2109
Epoch [78000/500000], Property: num_edges, Loss: 56631.0312
Epoch [79000/500000], Property: num_edges, Loss: 56122.3125
Epoch [80000/500000], Property: num_edges, Loss: 55631.3008
Epoch [81000/500000], Property: num_edges, Loss: 55156.8008
Epoch [82000/500000], Property: num_edges, Loss: 54698.0000
Epoch [83000/500000], Property: num_edges, Loss: 54254.5078
Epoch [84000/500000], Property: num_edges, Loss: 53826.1016
Epoch [85000/500000], Property: num_edge

...

Epoch [158000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [159000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [160000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [161000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [162000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [163000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [164000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [165000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [166000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [167000/500000], Property: number_of_nodes_in_the_largest_

Epoch [237000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [238000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [239000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [240000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [241000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [242000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [243000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [244000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [245000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [246000/500000], Property: number_of_nodes_in_the_largest_

Epoch [316000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [317000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [318000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [319000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [320000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [321000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [322000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [323000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [324000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [325000/500000], Property: number_of_nodes_in_the_largest_

Epoch [395000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [396000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [397000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [398000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [399000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [400000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [401000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [402000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [403000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [404000/500000], Property: number_of_nodes_in_the_largest_

Epoch [474000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [475000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [476000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [477000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [478000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [479000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [480000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [481000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [482000/500000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 6.9930
Epoch [483000/500000], Property: number_of_nodes_in_the_largest_

In [ ]:
# check train_features shape, ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (40,) + inhomogeneous part.
# check train_features[0] shape, (40,) -> it should be (40,1)
# check train_features[0][0] shape, (1,) -> it should be (1,1)

#print(train_features[0].shape)

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/FC_matrice_GCN_test_R2_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/FC_matrice_GCN_train_R2_plot.png', dpi=300, bbox_inches='tight')


# Test with new function that has limited convergence to see if it impacts the test R2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv.pkl", "wb") as f:
    pkl.dump(results, f)

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/FC_matrice_GCN_test_R2_plot_limited_cv.png', dpi=300, bbox_inches='tight')

In [ ]:
# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/FC_matrice_GCN_train_R2_plot_limited_cv.png', dpi=300, bbox_inches='tight')

#### Test with more properties

In [ ]:
import networkx as nx
import numpy as np

def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Use the average path length of the largest connected component for disconnected graphs
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    
def betweenness_centralization(G):
    n = len(G)
    betweenness = nx.betweenness_centrality(G)
    max_betweenness = max(betweenness.values())
    centralization = sum(max_betweenness - bet for bet in betweenness.values())
    if n > 2:
        centralization /= (n - 1) * (n - 2) / 2
    return centralization

def pagerank_centralization(G, alpha=0.85):
    n = len(G)
    pagerank = nx.pagerank(G, alpha=alpha)
    max_pagerank = max(pagerank.values())
    centralization = sum(max_pagerank - pr for pr in pagerank.values())
    if n > 1:
        centralization /= (n - 1)
    return centralization

def clustering_properties(G):
    average_clustering = nx.average_clustering(G)
    transitivity = nx.transitivity(G)
    return average_clustering, transitivity

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        
        # Number of nodes
        num_nodes = G.number_of_nodes()
        
        # Number of edges
        num_edges = G.number_of_edges()
        
        # Density
        density = nx.density(G)
        
        # Average Path Length
        avg_path_len = calculate_avg_path_length(G)
        
        # Diameter
        if nx.is_connected(G):
            diameter = nx.diameter(G)
        else:
            # Use the diameter of the largest connected component for disconnected graphs
            components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
            largest_component = max(components, key=len)
            diameter = nx.diameter(largest_component)
        
        # Radius
        if nx.is_connected(G):
            radius = nx.radius(G)
        else:
            radius = nx.radius(largest_component)
        
        # Clustering Coefficient
        clustering_coeff = nx.average_clustering(G)
        
        # Transitivity
        transitivity = nx.transitivity(G)
        
        # Assortativity
        assortativity = nx.degree_assortativity_coefficient(G)
        
        # Number of Cliques
        num_cliques = len(list(nx.find_cliques(G)))
        
        # Number of Triangles
        num_triangles = sum(nx.triangles(G).values()) / 3
        
        # Number of Squares (4-cycles)
        num_squares = sum(nx.square_clustering(G).values()) / 4
        
        # Size of the Largest Connected Component
        largest_component_size = len(max(nx.connected_components(G), key=len))
        
        # Average Degree
        degrees = [d for n, d in G.degree()]
        avg_degree = np.mean(degrees)
        
        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        avg_betweenness_centrality = np.mean(list(betweenness_centrality.values()))
        
        # Eigenvalues of the Adjacency Matrix (for spectral properties)
        eigenvalues = np.linalg.eigvals(nx.adjacency_matrix(G).todense())
        spectral_radius = max(eigenvalues)
        algebraic_connectivity = sorted(eigenvalues)[1]  # second smallest eigenvalue
        
        # Graph Laplacian Eigenvalues
        laplacian_eigenvalues = np.linalg.eigvals(nx.laplacian_matrix(G).todense())
        graph_energy = sum(abs(laplacian_eigenvalues))
        
        # Small-World-ness
        # Compare clustering coefficient and average path length with those of a random graph
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)
        random_clustering_coeff = nx.average_clustering(random_graph)
        random_avg_path_len = calculate_avg_path_length(random_graph)
        small_world_coefficient = (clustering_coeff / random_clustering_coeff) / (avg_path_len / random_avg_path_len)

        # Calculate Betweenness Centralization
        betweenness_cent = betweenness_centralization(G)
        print(f"Betweenness Centralization: {betweenness_cent}")

        # Calculate PageRank Centralization
        pagerank_cent = pagerank_centralization(G)
        print(f"PageRank Centralization: {pagerank_cent}")

        # Calculate Clustering properties
        avg_clustering, transitivity = clustering_properties(G)
        print(f"Average Clustering Coefficient: {avg_clustering}")
        print(f"Transitivity: {transitivity}")
        
        properties.append({
            "num_nodes": num_nodes,
            "num_edges": num_edges,
            "density": density,
            "avg_path_len": avg_path_len,
            "diameter": diameter,
            "radius": radius,
            "clustering_coeff": clustering_coeff,
            "transitivity": transitivity,
            "assortativity": assortativity,
            "num_cliques": num_cliques,
            "num_triangles": num_triangles,
            "num_squares": num_squares,
            "largest_component_size": largest_component_size,
            "avg_degree": avg_degree,
            "avg_betweenness_centrality": avg_betweenness_centrality,
            "spectral_radius": spectral_radius,
            "algebraic_connectivity": algebraic_connectivity,
            "graph_energy": graph_energy,
            "small_world_coefficient": small_world_coefficient
        })
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties_long = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties_long = compute_graph_properties(selected_dataset)

#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "wb") as f:
    pkl.dump(train_properties_long, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "wb") as f:
    pkl.dump(test_properties_long, f)

#load the properties
# with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "rb") as f:
#     train_properties_long = pkl.load(f)

# with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "rb") as f:
#     test_properties_long = pkl.load(f)

In [ ]:
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']

#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv_long.pkl", "wb") as f:
    pkl.dump(results, f)

In [ ]:

colors_long = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_names_long)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/FC_matrice_GCN_test_R2_plot_limited_cv_long.png', dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_names_long)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)

    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/FC_matrice_GCN_train_R2_plot_limited_cv_long.png', dpi=300, bbox_inches='tight')
    